In [1]:
## import packages
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from imutils import paths
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, save_model, Sequential
from tensorflow.keras.applications import ResNet50V2, VGG16
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [2]:
## data preprocessing
imagePath = "./all/train"
imagePaths = list(paths.list_images(imagePath))
data = []
labels = []

for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (256, 256))
    data.append(image)
    labels.append(label)

data = np.array(data) / 255.0
labels = np.array(labels)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)
labels = to_categorical(integer_encoded)

(x_train, x_val, y_train, y_val) = train_test_split(data, labels, test_size=0.20, stratify=labels)

In [ ]:
## AI model training

model = Sequential()
adam_s = Adam(learning_rate = 0.00001)

#model.add(VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet', pooling='average'))
model.add(ResNet50V2(input_shape=(256, 256, 3),include_top=False, weights='imagenet',pooling='average'))

for layer in model.layers:
    layer.trainable = False

model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=adam_s, metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 8, 8, 2048)        23564800  
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 8, 2048)        8192      
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               16777344  
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387       
Total params: 40,367,235
Trainable params: 16,798,339
Non-trainable params: 23,568,896
___________________________________

In [3]:
H = model.fit(x_train, y_train, epochs=15, validation_data=(x_val, y_val), batch_size=8)
#H = model.fit(data, labels, epochs=50, validation_split=0.2, batch_size=8)

Epoch 1/15
20/20 [==============================] - 29s 1s/step - loss: 0.9353 - accuracy: 0.5750 - val_loss: 0.8923 - val_accuracy: 0.6750
Epoch 2/15
20/20 [==============================] - 28s 1s/step - loss: 0.1247 - accuracy: 1.0000 - val_loss: 0.6303 - val_accuracy: 0.7500
Epoch 3/15
20/20 [==============================] - 28s 1s/step - loss: 0.0408 - accuracy: 1.0000 - val_loss: 0.4896 - val_accuracy: 0.8000
Epoch 4/15
20/20 [==============================] - 28s 1s/step - loss: 0.0164 - accuracy: 1.0000 - val_loss: 0.4287 - val_accuracy: 0.8250
Epoch 5/15
20/20 [==============================] - 27s 1s/step - loss: 0.0102 - accuracy: 1.0000 - val_loss: 0.3902 - val_accuracy: 0.8250
Epoch 6/15
20/20 [==============================] - 28s 1s/step - loss: 0.0081 - accuracy: 1.0000 - val_loss: 0.3634 - val_accuracy: 0.8750
Epoch 7/15
20/20 [==============================] - 28s 1s/step - loss: 0.0055 - accuracy: 1.0000 - val_loss: 0.3479 - val_accuracy: 0.8750
Epoch 8/15
20/20 [==

In [5]:
## save AI model
save_model(model, "covid/1")

INFO:tensorflow:Assets written to: covid/1/assets


In [8]:
!pip install tqdm
print("[INFO] loading untrained test images...")
imagePathTest = "./all/test"
imagePathsTest = list(paths.list_images(imagePathTest))
print(len(imagePathsTest))

dataTest = []
labelsTest = []

# loop over the image paths
for imagePathTest in imagePathsTest:

    # extract the class label from the filename
	  labelTest = imagePathTest.split(os.path.sep)[-2]
  
	  # load the image, swap color channels, and resize it to be a fixed
	  # 256x256 pixels while ignoring aspect ratio
	  imageTest = cv2.imread(imagePathTest)
	  imageTest = cv2.cvtColor(imageTest, cv2.COLOR_BGR2RGB)
	  imageTest = cv2.resize(imageTest, (256, 256))

	  # update the data and labels lists, respectively
	  dataTest.append(imageTest)
	  labelsTest.append(labelTest)
# convert the data and labels to NumPy arrays while scaling the pixel
# intensities to the range [0, 255]
dataTest = np.array(dataTest) / 255.0
labelsTest = np.array(labelsTest)


# perform one-hot encoding on the labels
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labelsTest)
labelsTest = to_categorical(integer_encoded)

print(labelsTest)

[INFO] loading untrained test images...
27
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [ ]:
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
import cv2
import json
import requests
from tqdm import tqdm

In [41]:
#get some sample data
sample_test_data = dataTest[2]
sample_test_labels = labelsTest[2]

print(sample_test_data.shape)

(256, 256, 3)


In [42]:
class_names = ['covid', 'normal', 'pneumonia_bac']

# pre-process data 
#sample_test_data_processed = np.expand_dims(sample_test_data / 255., axis=3)

sample_test_data = sample_test_data.reshape(-1,256,256,3)
print(sample_test_data.shape)

(1, 256, 256, 3)


In [43]:
model.predict(sample_test_data)

array([[7.0699543e-04, 4.7419232e-04, 9.9881881e-01]], dtype=float32)

In [51]:
data = json.dumps({"content-type": "application/json", "signature_name": "serving_default", 
                   "instances": sample_test_data.tolist()})

TypeError: dump() missing 1 required positional argument: 'fp'

In [50]:
# create payload
#headers = {"content-type": "application/json"}
data = json.dumps({"content-type": "application/json", "signature_name": "serving_default", 
                   "instances": sample_test_data.tolist()})

json_response = requests.post('http://127.0.0.1:8501/v1/models/covid:predict', data=data)
predictions = json.loads(json_response.text)['predictions']
print(predictions[0])

[0.00070699543, 0.000474192319, 0.998818815]


In [52]:
with open("please.json", "w") as text_file:
    text_file.write("%s" % data)

In [53]:
!curl 127.0.0.1:8501/v1/models/covid:predict -X POST -d@please.json

{
    "predictions": [[0.00070699543, 0.000474192319, 0.998818815]
    ]
}